In [1]:
from keras.models import Sequential, Model
from keras.layers import Dense, Input, concatenate, Dropout, Reshape, Conv1D, MaxPooling1D, Flatten, Activation
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam, Adadelta
from keras import backend as K
from keras.losses import mean_absolute_percentage_error
from keras.callbacks import ModelCheckpoint, EarlyStopping

import sys
from ZernikeCNN import ZernikeConv, resampleGraph_expand, Normalized_resampleGraph_expand, ZernikeDecomp
import numpy as np
import os
import scipy.io as sio
import scipy.stats


Using TensorFlow backend.


In [2]:
import scipy.io as sio

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = sio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], sio.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, sio.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        else:
            dict[strg] = elem
    return dict


In [3]:
def load_patchfeature_input(path):
    ZerNet_preproc = loadmat(path)
    Input_Patch_Features = ZerNet_preproc['ZerNet_preproc']['scale_1']['ZerPatch']['FeaVec'].astype('float32')
    return Input_Patch_Features

def load_mapping_infor(path):
    sampled_infor = loadmat(path)
    mesh_faces = (sampled_infor['sampled_surface']['F']-1).astype('int32')
    resample_graph_I = (sampled_infor['sampled_surface']['I']-1).astype('int32')
    resample_graph_B = sampled_infor['sampled_surface']['B'].astype('float32')
    return mesh_faces, resample_graph_I, resample_graph_B

def load_output_stress(path):
    sampled_infor = loadmat(path)
    stress_out = (sampled_infor['sampled_surface']['J2m']*1000).astype('float32')
    return stress_out

def load_Zerpatch_input(path, scalename):
    ZerNet_preproc = loadmat(path)
    Zerbases = ZerNet_preproc['ZerNet_preproc'][scalename]['ZerPatch']['bases'].astype('float32')
    resampleGraph = (ZerNet_preproc['ZerNet_preproc'][scalename]['ZerPatch']['resamplePids']-1).astype('int32')
    scale ={'Zerbases':Zerbases,
            'resampleGraph':resampleGraph}
    return scale

def create_data(patch_path, sampled_path, argument=False):
    scale_1 = load_Zerpatch_input(patch_path, 'scale_1')
    Zerbases = scale_1['Zerbases']
    resampleGraph = scale_1['resampleGraph']
    Disk_Feature = load_patchfeature_input(patch_path)  
    stress_out = load_output_stress(sampled_path)
    return Disk_Feature,resampleGraph,Zerbases,stress_out

In [4]:
import keras
import numpy as np
import os
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_model_names, patches_folder, sampled_folder, argument=False, batch_size=1, cut_num=65, basis_num = 21, 
                 num_input_channels=3, shuffle=True):
        'Initialization'
        self.list_model_names = list_model_names
        self.patches_folder = patches_folder
        self.sampled_folder = sampled_folder
        self.argument = argument
        self.cut_num = cut_num
        self.basis_num = basis_num
        self.batch_size = batch_size
        self.num_input_channels = num_input_channels
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_model_names) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index:index+1]

        # Find list of model_names
        list_model_names_temp = [self.list_model_names[k] for k in indexes]

        # Generate data
        X, Y = self.__data_generation(list_model_names_temp)

        return X, Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_model_names))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)        

    def __data_generation(self, list_model_names_temp):
        'Generates data containing batch_size samples'        
        model_name = list_model_names_temp[0]

        # Generate data
        patch_path = os.path.join(self.patches_folder,model_name)
        sampled_path = os.path.join(self.sampled_folder,model_name)
            
        Disk_Feature,resampleGraph,Zerbases,stress_out = create_data(patch_path,sampled_path,self.argument)
        mesh_faces, resample_graph_I, resample_graph_B = load_mapping_infor(sampled_path)
        
        x_Disk_Feature = np.expand_dims(Disk_Feature,axis=0)
        x_rG = np.expand_dims(resampleGraph,axis=0)
        x_Zb = np.expand_dims(Zerbases,axis=0)
        
        x_F = np.expand_dims(mesh_faces,axis=0)
        x_I = np.expand_dims(resample_graph_I,axis=0)
        x_B = np.expand_dims(resample_graph_B,axis=0)
        
        X = [x_Disk_Feature, x_rG, x_F, x_I, x_B, x_Zb]
        Y = np.expand_dims(stress_out, axis=-1)
        Y = np.expand_dims(Y, axis=0)
        
        return X, Y

In [5]:
def Initial_ZerConv_block(nf, nrays_expand, x_Disk_Feature, x_Zb, dropout_ratio=0.25):
    Zercoeff = ZernikeDecomp(angular_axis=False)([x_Disk_Feature,x_Zb])
    Features = ZernikeConv(filters = nf, numofrays = nrays_expand, angular_axis = False, 
                           angular_expand=True,angular_pooling=False, activation='relu')(Zercoeff)
    Features = Dropout(dropout_ratio)(Features)
    return Features

def ZerConv_block(nf,nrays_in,x_Feature, x_rG, x_F, x_I, x_B, x_Zb, pooling=False, dropout_ratio=0.25):
    Disk_Features = resampleGraph_expand(angular_axis=True)([x_Feature, x_rG, x_F, x_I, x_B])
    Zercoeff = ZernikeDecomp(angular_axis=True, numofrays=nrays_in)([Disk_Features,x_Zb])
    Features = ZernikeConv(filters = nf, numofrays = nrays_in, angular_axis = True, 
                           angular_expand=False,angular_pooling=pooling, activation='relu')(Zercoeff)  
    Features = Dropout(dropout_ratio)(Features)
    return Features

In [6]:
def ZerNet(nrays, numofresampledpoints, cut_num, basis_num = 21, num_input_channels=3):
    
    Disk_Features_in = Input(shape=(None,cut_num,num_input_channels,), dtype = 'float32')
    resampleGraph = Input(shape=(None,cut_num,), dtype = 'int32', name = 'rG')
    F = Input(shape=(None,3), dtype = 'int32', name = 'F')
    I = Input(shape=(numofresampledpoints,), dtype = 'int32', name = 'I')
    B = Input(shape=(numofresampledpoints,3,), dtype = 'float32', name = 'B')
    Zerbases = Input(shape=(None,cut_num,basis_num,), dtype = 'float32', name = 'Zb')

    Features = Initial_ZerConv_block(128,nrays,Disk_Features_in, Zerbases)
    Features = ZerConv_block(256, nrays, Features, resampleGraph, F, I, B, Zerbases, pooling=False)
    Features = ZerConv_block(512, nrays, Features, resampleGraph, F, I, B, Zerbases, pooling=True)

    Features = Conv1D(filters = 800, kernel_size=1, activation='relu')(Features)
    Features = Dropout(0.25)(Features)

    Features = Conv1D(filters = 1, kernel_size=1, activation='linear')(Features)

    model = Model(inputs = [Disk_Features_in,resampleGraph, F, I, B, Zerbases], outputs = Features)
    
    return model

In [7]:
def mae_percentage_acc(y_true, y_pred):
    acc = 100 - mean_absolute_percentage_error(y_true, y_pred)
    return acc

In [9]:
# Datasets
patches_folder = './Data/Aneurysm Dataset/ZerNet Input/Input ZerPatches' 
sampled_folder = './Data/Aneurysm Dataset/UniformSampling_surfaces'

model_names = os.listdir(patches_folder)

# print(model_names)

In [10]:
# Parameters
params = {'batch_size': 1,
          'cut_num': 65,
          'basis_num': 21,
          'num_input_channels': 3,
          'shuffle': True}

test_model_names = ['TP_Ia166I.mat']
train_model_names = list(set(model_names)-set(test_model_names))
# Generators
training_generator = DataGenerator(train_model_names, patches_folder, sampled_folder, argument=False, **params)
validation_generator = DataGenerator(test_model_names, patches_folder, sampled_folder, argument=False, **params)
train_steps = len(train_model_names)
val_steps = len(test_model_names)

In [11]:
num_input_channels = 3
cut_num = 65
basis_num = 21
numofresampledpoints = 8000
nrays = 1 # nrays = 4

model = ZerNet(nrays, numofresampledpoints, cut_num, basis_num, num_input_channels)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 65, 3)   0                                            
____________________________________________________________________________________________________
Zb (InputLayer)                  (None, None, 65, 21)  0                                            
____________________________________________________________________________________________________
zernike_decomp_1 (ZernikeDecomp) (None, None, 21, 3)   0           input_1[0][0]                    
                                                                   Zb[0][0]                         
____________________________________________________________________________________________________
zernike_conv_1 (ZernikeConv)     (None, None, 1, 128)  8192        zernike_decomp_1[0][0]  

In [ ]:
trained_model_name = 'ZerNet_aneury_'+ test_model_names[0]
weights_dir = './Train Models/aneurysm_train_checkpoints'

# model.compile(loss= 'mean_absolute_error', optimizer = adam, metrics=['mse',mae_percentage_acc])

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss= 'mean_absolute_error', optimizer = adam, metrics=['mse',mae_percentage_acc])
checkpointer = ModelCheckpoint(filepath= os.path.join(weights_dir, trained_model_name), 
                               monitor = 'val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
earlystopping = EarlyStopping(monitor='val_mean_squared_error', min_delta=0, patience=30, verbose=0, mode='min')


network_history = model.fit_generator(generator=training_generator, steps_per_epoch=train_steps,
                                      epochs=200, verbose=1,
                                      callbacks=[checkpointer, earlystopping],
                                      validation_data=validation_generator,
                                      validation_steps=val_steps)

In [20]:
def load_test_data(patch_folder, sampled_folder, model_name):
    patch_path = os.path.join(patch_folder, model_name)
    sampled_path = os.path.join(sampled_folder, model_name)

    Disk_Feature,resampleGraph,Zerbases,stress_out = create_data(patch_path,sampled_path)
    mesh_faces, resample_graph_I, resample_graph_B = load_mapping_infor(sampled_path)
    
    x_Disk_Feature = np.expand_dims(Disk_Feature,axis=0)
    x_rG = np.expand_dims(resampleGraph,axis=0)
    x_Zb = np.expand_dims(Zerbases,axis=0)
        
    x_F = np.expand_dims(mesh_faces,axis=0)
    x_I = np.expand_dims(resample_graph_I,axis=0)
    x_B = np.expand_dims(resample_graph_B,axis=0)
        
    X = [x_Disk_Feature, x_rG, x_F, x_I, x_B, x_Zb]
    Y = np.expand_dims(stress_out, axis=-1)
    Y = np.expand_dims(Y, axis=0)
    
    return X, Y

In [21]:
def mae_acc(yp,y_true):
    return (1-np.sum(np.abs(yp - y_true)/y_true)/len(y_true))*100

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

weights_dir = './Train Models/aneurysm_train_checkpoints'
predict_folder = './Data/Aneurysm Dataset/ZerNet Predict'

model_name = 'TP_Ia166I.mat'
model = ZerNet(nrays, numofresampledpoints, cut_num, basis_num, num_input_channels)
trained_model_name = 'ZerNet_aneury_'+ model_name

model.load_weights(os.path.join(weights_dir, trained_model_name))

test_X, test_Y = load_test_data(patches_folder, sampled_folder, model_name)
Yp = model.predict(test_X, batch_size=1).flatten()
Y = test_Y.flatten()
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(Y, Yp)

plt.figure()
plt.xlim(0,220)
plt.ylim(0,220)

plt.scatter(Y, Yp)
print(r_value)

Acc = mae_acc(Yp,Y)
print(Acc)

sio.savemat(os.path.join(predict_folder, model_name),{'predict_J2m':Yp})